### CSV And Excel files -Structured Data

In [5]:
import pandas as pd
import os

In [2]:
os.makedirs("data/structured_files",exist_ok=True)

In [6]:
# Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

df = pd.DataFrame(data)
df.to_csv('data/structured_files/products.csv', index=False)

In [7]:
# Save as Excel with multiple sheets
with pd.ExcelWriter('data/structured_files/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)

    # Add another sheet
    summary_data = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

### CSV Processing

In [8]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

In [9]:
# Method 1: CSVLoader - Each row becomes a document
print("1️⃣ CSVLoader - Row-based Documents")
csv_loader = CSVLoader(
    file_path='data/structured_files/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    }
)

csv_docs = csv_loader.load()
print(f"Loaded {len(csv_docs)} documents (one per row)")
print(csv_docs)
print("\nFirst document:")
print(f"Content: {csv_docs[0].page_content}")
print(f"Metadata: {csv_docs[0].metadata}")

1️⃣ CSVLoader - Row-based Documents
Loaded 5 documents (one per row)
[Document(metadata={'source': 'data/structured_files/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4K monitor with HDR support'), Document(metadata={'source': 'data/structured_files/produ

In [15]:
from typing import List
from langchain_core.documents import Document
import pandas as pd

# Method 2: Custom CSV processing for better control
print("\n2️⃣ Custom CSV Processing")

def process_csv_intelligently(filepath: str) -> List[Document]:
    """Process CSV with intelligent document creation"""
    df = pd.read_csv(filepath)
    documents = []

    # Strategy 1: One document per row with structured content
    for idx, row in df.iterrows():
        # Create structured content
        content = f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""
        
       # Create document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                'data_type': 'product_info'
            }
        )
        documents.append(doc)
    return documents


2️⃣ Custom CSV Processing


In [16]:
process_csv_intelligently('data/structured_files/products.csv')

[Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99\n        Stock: 50 units\n        Description: High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'price': 29.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99\n        Stock: 200 units\n        Description: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'price': 79.99, 'data_type': 'product_info'}, page_content='Product Informa

In [17]:
# 📊 CSV Processing Strategies
print("\n📊 CSV Processing Strategies:")
print("\n1. Row-based (CSVLoader):")
print("   ✅  Simple one-row-one-document")
print("   ✅  Good for record lookups")
print("   ❌  Loses table context")

print("\n2. Intelligent Processing:")
print("   ✅  Preserves relationships")
print("   ✅  Creates summaries")
print("   ✅  Rich metadata")
print("   ✅  Better for Q&A")


📊 CSV Processing Strategies:

1. Row-based (CSVLoader):
   ✅  Simple one-row-one-document
   ✅  Good for record lookups
   ❌  Loses table context

2. Intelligent Processing:
   ✅  Preserves relationships
   ✅  Creates summaries
   ✅  Rich metadata
   ✅  Better for Q&A


### Excel Processing

In [22]:
import pandas as pd
from typing import List
from langchain_core.documents import Document

# Method 3: Pandas for Multi-sheet Excel
print("\n3️⃣ Pandas for Multi-sheet Excel")

def process_excel_with_pandas(filepath: str) -> List[Document]:
    """Process Excel file with multiple sheets"""
    # Read all sheets (sheet_name=None returns a dictionary of DataFrames)
    xls_dict = pd.read_excel(filepath, sheet_name=None)
    documents = []

    for sheet_name, df in xls_dict.items():
        print(f"Processing sheet: {sheet_name} ({len(df)} rows)")

        # Handle different sheets differently
        for idx, row in df.iterrows():
            if sheet_name == 'Products':
                content = f"Product: {row['Product']}\nCategory: {row['Category']}\nPrice: ${row['Price']}"
            
            elif sheet_name == 'Summary':
                content = f"Summary for {row['Category']}:\nTotal Items: {row['Total_Items']}\nTotal Value: ${row['Total_Value']}"
            
            else:
                continue # Skip unknown sheets

            # Create document with sheet metadata
            doc = Document(
                page_content=content,
                metadata={
                    'source': filepath,
                    'sheet': sheet_name,
                    'row': idx,
                    'type': 'excel_record'
                }
            )
            documents.append(doc)

    return documents



3️⃣ Pandas for Multi-sheet Excel


In [23]:
excel_docs = process_excel_with_pandas('data/structured_files/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")

Processing sheet: Products (5 rows)
Processing sheet: Summary (2 rows)
Processed 7 sheets


In [30]:
excel_docs

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet': 'Products', 'row': 0, 'type': 'excel_record'}, page_content='Product: Laptop\nCategory: Electronics\nPrice: $999.99'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet': 'Products', 'row': 1, 'type': 'excel_record'}, page_content='Product: Mouse\nCategory: Accessories\nPrice: $29.99'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet': 'Products', 'row': 2, 'type': 'excel_record'}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: $79.99'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet': 'Products', 'row': 3, 'type': 'excel_record'}, page_content='Product: Monitor\nCategory: Electronics\nPrice: $299.99'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet': 'Products', 'row': 4, 'type': 'excel_record'}, page_content='Product: Webcam\nCategory: Electronics\nPrice: $89.99'),
 Document(m

In [31]:
from langchain_community.document_loaders import UnstructuredExcelLoader

# Method 4: Unstructured Excel Loader (Using "unstructured" library)
print("\n4️⃣ UnstructuredExcelLoader")

try:
    loader = UnstructuredExcelLoader(
        "data/structured_files/inventory.xlsx", 
        mode="elements"
    )
    docs = loader.load()

    print(f"Loaded {len(docs)} documents")
    print(f"First document content: {docs[0].page_content}")
    print(f"Metadata: {docs[0].metadata}")

except Exception as e:
    print(f"Error loading Excel file: {e}")


4️⃣ UnstructuredExcelLoader
Error loading Excel file: No module named 'msoffcrypto'
